In [7]:
import sympy
x = sympy.Symbol('x')
N = 25
exact = [float(sympy.integrate(x**n * sympy.exp(1 - x), (x, 0, 1))) for n in range(N)]

Для вычисления интегралов используем рекуррентную формулу $I_n = nI_{n-1} - 1$, причем $I_0 = e - 1$. В коде ниже $I_0 \equiv I_{n-1}$, $I \equiv I_n$.

In [8]:
def upwards_recursion(n):
    """Compute the integrals using the upwards recursion."""
    from math import exp
    I_0 = exp(1) - 1
    I_values = [I_0]
    for i in range(1, n):
        I = I_0 * i - 1
        I_values.append(I)
        I_0, I = I, I
    return I_values

In [9]:
values = upwards_recursion(25)
for value, exact_value in zip(values, exact):
    print(value, exact_value)

from numpy.testing import assert_allclose
assert_allclose(values, exact)

1.718281828459045 1.7182818284590453
0.7182818284590451 0.7182818284590452
0.4365636569180902 0.43656365691809046
0.30969097075427054 0.30969097075427143
0.23876388301708218 0.23876388301708565
0.1938194150854109 0.19381941508542824
0.16291649051246537 0.16291649051256946
0.1404154335872576 0.14041543358798622
0.12332346869806088 0.12332346870388973
0.1099112182825479 0.10991121833500754
0.09911218282547907 0.0991121833500754
0.09023401108026974 0.09023401685082952
0.08280813296323686 0.08280820220995427
0.07650572852207915 0.07650662872940558
0.07108019930910814 0.07109280221167809
0.06620298963662208 0.0663920331751714
0.05924783418595325 0.06227253080274239
0.007213181161205284 0.05863302364662064
-0.8701627390983049 0.05539442563917152
-17.533092042867793 0.052494087144258815
-351.66184085735586 0.049881742885176245
-7385.898658004473 0.04751660058870116
-162490.7704760984 0.04536521295142559
-3737288.7209502636 0.04339989788278872
-89694930.30280632 0.041597549186929206


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 52%
Max absolute difference: 89694930.34440386
Max relative difference: 2.15625517e+09
 x: array([ 1.718282e+00,  7.182818e-01,  4.365637e-01,  3.096910e-01,
        2.387639e-01,  1.938194e-01,  1.629165e-01,  1.404154e-01,
        1.233235e-01,  1.099112e-01,  9.911218e-02,  9.023401e-02,...
 y: array([1.718282, 0.718282, 0.436564, 0.309691, 0.238764, 0.193819,
       0.162916, 0.140415, 0.123323, 0.109911, 0.099112, 0.090234,
       0.082808, 0.076507, 0.071093, 0.066392, 0.062273, 0.058633,...

Операция вычитания в машинной арифметике способствует накоплению погрешности. Значение вычисляемого интеграла при любом n обазяно быть положительным, однако из-за погрешности на некоторой итерации последовательность ушла в отрицательные значения. Из-за соответствующего вида рекуррентной формулы, вернуться к положительным значениям на следующих итерациях невозможно, а по модулю величина вычисляемого интеграла будет только расти. Нужно искать другой метод вычисления этих интегралов, например, идти не снизу вверх, а сверху вниз. 

Заметим, что $I_n \rightarrow 0$, $n \rightarrow \infty$. Тогда выберем $I_{24} = 0$, то есть будем спускаться вниз от 0 по формуле $I_{n-1} = \frac{I_n + 1}{i}$. В коде ниже $I_0 \equiv I_n$, $I \equiv I_{n-1}$.

In [4]:
def downwards_recursion(n):
    I_0 = 0
    I_values = [I_0]
    for i in range(n-1, 0, -1):
        I = (I_0 + 1) / i
        I_values.append(I)
        I_0, I = I, I
    return list(reversed(I_values))

In [5]:
values = downwards_recursion(25)

for value, exact_value in zip(values, exact):
    print(value, exact_value)

from numpy.testing import assert_allclose
assert_allclose(values, exact)

1.718281828459045 1.7182818284590453
0.7182818284590452 0.7182818284590452
0.43656365691809046 0.43656365691809046
0.30969097075427143 0.30969097075427143
0.23876388301708565 0.23876388301708565
0.19381941508542824 0.19381941508542824
0.16291649051256946 0.16291649051256946
0.14041543358798622 0.14041543358798622
0.12332346870388972 0.12332346870388973
0.10991121833500754 0.10991121833500754
0.09911218335007542 0.0991121833500754
0.09023401685082953 0.09023401685082952
0.08280820220995425 0.08280820220995427
0.07650662872940517 0.07650662872940558
0.07109280221167226 0.07109280221167809
0.06639203317508373 0.0663920331751714
0.06227253080133964 0.06227253080274239
0.05863302362277381 0.05863302364662064
0.055394425209928634 0.05539442563917152
0.05249407898864421 0.052494087144258815
0.04988157977288413 0.049881742885176245
0.047513175230566536 0.04751660058870116
0.04528985507246377 0.04536521295142559
0.041666666666666664 0.04339989788278872
0 0.041597549186929206


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 24%
Max absolute difference: 0.04159755
Max relative difference: 1.
 x: array([1.718282, 0.718282, 0.436564, 0.309691, 0.238764, 0.193819,
       0.162916, 0.140415, 0.123323, 0.109911, 0.099112, 0.090234,
       0.082808, 0.076507, 0.071093, 0.066392, 0.062273, 0.058633,...
 y: array([1.718282, 0.718282, 0.436564, 0.309691, 0.238764, 0.193819,
       0.162916, 0.140415, 0.123323, 0.109911, 0.099112, 0.090234,
       0.082808, 0.076507, 0.071093, 0.066392, 0.062273, 0.058633,...

Такой способ вычисления членов последовательности интегралов показал себя значительно лучше. Здесь мы обошли операцию вычитания, тем самым избежав лишних погрешностей и возможности случайно уйти в отрицательные значения.